# KOD DLA SIECI NEURONOWEJ 3 8 1

# Funkcje oraz klasy sieci

In [24]:
import numpy as np 

def relu(x):
        return x * (x > 0)

def relu_derivative(x):
    return 1. * (x > 0)

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)


#Klasa dla sieci 3-8-4-1
class NeuralNetwork:

    #Inicjalizacja sieci
    def __init__(self, x, y, seed, eta):
        np.random.seed(seed)
        self.input = x.T
        # Wagi z 3 wejsc do warstwy ukrytej z 8 neuronami
        self.weights1 = np.random.rand(self.input.shape[0],8).T
        #print(self.weights1)
        # Wagi z 1 warstwy ukrytej do drugiej warstwy ukrytej
        self.weights2 = np.random.rand(8,4).T
        #print(self.weights2)
        # Wagi z 4 neuronow drugiej warstwy ukrytej do 1 neuronu wyjsciowego
        self.weights3 = np.random.rand(4,1).T
        # Wartosci wyjsciowe        
        self.y = y.T
        self.output = np.zeros(self.y.shape)
        #print(self.output)
        self.eta = eta

    #Feedforward z funkcjami aktywacji jako argumenty dla tworzenia kombinacji
    def feedforward(self, activation_function_1, activation_function_2, activation_function_3):
        self.layer1 = activation_function_1(np.dot(self.weights1, self.input))
        self.layer2 = activation_function_2(np.dot(self.weights2, self.layer1))
        self.output = activation_function_3(np.dot(self.weights3, self.layer2))

    #Propagacja wsteczna z funkcjami aktywacji jako argumenty dla tworzenia kombinacji
    def backprop(self, derivative_activation_function_1, derivative_activation_function_2, derivative_activation_function_3):
        # Blad dla warstwy wyjsciowej
        delta_3 = (self.y - self.output) * derivative_activation_function_3(self.output)
        d_weights3 = self.eta * np.dot(delta_3, self.layer2.T)
        # Blad dla 2 warstwy ukrytej
        delta_2 = derivative_activation_function_2(self.layer2) * np.dot(self.weights3.T, delta_3)
        d_weights2 = self.eta * np.dot(delta_2, self.layer1.T)
        # Blad dla 1 warstwy ukrytej
        delta_1 = derivative_activation_function_1(self.layer1) * np.dot(self.weights2.T, delta_2)
        d_weights1 = self.eta * np.dot(delta_1, self.input.T)
        # Aktualizacja wag
        self.weights1 += d_weights1
        self.weights2 += d_weights2
        self.weights3 += d_weights3
    
    # Algorytm propagacji wstecznej wykonujacy sie przez kreslona liczbe epok
    def train(self, f1, f2,f3, epochs):
        activation_function_1 = relu
        derivative_activation_function_1 = relu_derivative
        activation_function_2 = relu
        derivative_activation_function_2 = relu_derivative
        activation_function_3 = relu
        derivative_activation_function_3 = relu_derivative

        if f1 == 'sigmoid':
            activation_function_1 = sigmoid
            derivative_activation_function_1 = sigmoid_derivative
        if f2 == 'sigmoid':
            activation_function_2 = sigmoid
            derivative_activation_function_2 = sigmoid_derivative
        if f3 == 'sigmoid':
            activation_function_3 = sigmoid
            derivative_activation_function_3 = sigmoid_derivative
        for _ in range(epochs):
            self.feedforward(activation_function_1, activation_function_2, activation_function_3)
            self.backprop(derivative_activation_function_1, derivative_activation_function_2, derivative_activation_function_3)

    #Algorytm propagacji wstecznej wykonujacy sie do osiagniecia warunku konca
    def train_cond(self, f1, f2,f3, cond):
        activation_function_1 = relu
        derivative_activation_function_1 = relu_derivative
        activation_function_2 = relu
        derivative_activation_function_2 = relu_derivative
        activation_function_3 = relu
        derivative_activation_function_3 = relu_derivative
        epochs = 0

        if f1 == 'sigmoid':
            activation_function_1 = sigmoid
            derivative_activation_function_1 = sigmoid_derivative
        if f2 == 'sigmoid':
            activation_function_2 = sigmoid
            derivative_activation_function_2 = sigmoid_derivative
        if f3 == 'sigmoid':
            activation_function_3 = sigmoid
            derivative_activation_function_3 = sigmoid_derivative
        while True:
            self.feedforward(activation_function_1, activation_function_2, activation_function_3)
            self.backprop(derivative_activation_function_1, derivative_activation_function_2, derivative_activation_function_3)
            epochs += 1
            if cond(epochs, self.output):
                break
        return epochs
    



# Funkcje do testowania

In [25]:
# Test do znajdowania kombinacji zwracajacej najlepsze wyniki dla roznych eta - iteruje przez seedy
def accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs):
    print("Test dokladnosci ")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([[1.,0.,0.,1.]] * len(combinations))   
        best_seeds = np.array([0] * len(combinations))  
        best_combinations = []

        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetwork(X, y, seed, eta)
                nn.train(comb[0], comb[1],comb[2], epochs)
                if abs(nn.output - expected).sum() < abs(results[combination_id] - expected).sum():
                    results[combination_id] = nn.output
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: abs(x[0] - expected).sum())

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " " + str(comb[2]) + " seed = " + str(best_seeds[comb_id]))
            print("Wynik: " + str(result) + " Blad: " + str(abs(result - expected).sum()))

def time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs):
    print("Test czasu")
    for eta in etas:
        print(" ")
        print("Dla eta = " + str(eta))
        results = np.array([0] * len(combinations))  
        best_seeds = np.array([0] * len(combinations))  
        best_combinations = []
        for combination_id in range(len(combinations)):
            comb = combinations[combination_id]
            for seed in range(min_seed, max_seed):
                nn = NeuralNetwork(X, y, seed, eta)
                ep = nn.train_cond(comb[0], comb[1], comb[2], lambda epochs, output: abs(output - expected).sum() < 0.1 or epochs > max_epochs)
                if(ep < results[combination_id] or results[combination_id] == 0):
                    results[combination_id] = ep
                    best_seeds[combination_id] = seed
            best_combinations.append((results[combination_id], combination_id))

        best_combinations.sort(key=lambda x: x[0])

        for i in range(len(best_combinations)):
            result = best_combinations[i][0]
            comb_id = best_combinations[i][1]
            comb = combinations[comb_id]
            print("Kombinacja: " + str(comb[0]) + " " + str(comb[1]) + " " + str(comb[2]) + " seed = " + str(best_seeds[comb_id]) + " epoki = " + str(result))

# Nauka XOR

In [26]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[1],[1],[0]])


Testy dokładności

In [27]:

max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu", "relu"], ["relu", "relu", "sigmoid"], ["sigmoid", "sigmoid", "sigmoid"], ["sigmoid", "sigmoid", "relu"], 
                         ["relu", "sigmoid", "sigmoid"], ["relu", "sigmoid", "relu"],  ["sigmoid", "relu", "sigmoid"], ["sigmoid", "relu", "relu"]])
expected = np.array([[0.,1.,1.,0.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min




Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 418
Wynik: [0.00976309 0.99717533 0.99736927 0.00113962] Blad: 0.01635811472464227
Kombinacja: relu relu sigmoid seed = 79
Wynik: [0.50277944 0.99069713 0.99358959 0.9999388 ] Blad: 1.5184315296448134
Kombinacja: relu sigmoid relu seed = 188
Wynik: [0.33782017 0.59608138 0.58169002 0.56167259] Blad: 1.721721357544927
Kombinacja: sigmoid relu relu seed = 92
Wynik: [0.412529   0.52906994 0.50387395 0.55615659] Blad: 1.9357416988284404
Kombinacja: relu sigmoid sigmoid seed = 220
Wynik: [0.71354227 0.76321488 0.75660417 0.76830909] Blad: 1.962032321766149
Kombinacja: sigmoid relu sigmoid seed = 259
Wynik: [0.82209761 0.86701566 0.86135624 0.89046259] Blad: 1.9841883052855795
Kombinacja: sigmoid sigmoid relu seed = 27
Wynik: [0.4768786  0.50591137 0.50460874 0.5203031 ] Blad: 1.9866615930204965
Kombinacja: sigmoid sigmoid sigmoid seed = 220
Wynik: [0.76244499 0.77061493 0.76852774 0.77352385] Blad: 1.9968261710916155
 
Dl

Testy czasu

In [28]:
max_epochs = 10000
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 418 epoki = 3531
Kombinacja: relu relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu relu seed = 1 epoki = 10001
 
Dla eta = 0.01
Kombinacja: relu relu relu seed = 147 epoki = 526
Kombinacja: relu sigmoid relu seed = 188 epoki = 3135
Kombinacja: sigmoid relu relu seed = 483 epoki = 4429
Kombinacja: relu relu sigmoid seed = 285 epoki = 8691
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.05
Kombinacja: relu sigmoid relu seed = 188 epoki = 614
Kombinac

# Nauka AND 

In [29]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[0],[0],[1]])

Testy dokladnosci

In [30]:
max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu", "relu"], ["relu", "relu", "sigmoid"], ["sigmoid", "sigmoid", "sigmoid"], ["sigmoid", "sigmoid", "relu"], 
                         ["relu", "sigmoid", "sigmoid"], ["relu", "sigmoid", "relu"],  ["sigmoid", "relu", "sigmoid"], ["sigmoid", "relu", "relu"]])
expected = np.array([[0.,0.,0.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 449
Wynik: [-0.          0.00149273  0.00114513  0.99843348] Blad: 0.004204375919910075
Kombinacja: relu relu sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid sigmoid relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: relu sigmoid sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: relu sigmoid relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid relu sigmoid seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
Kombinacja: sigmoid relu relu seed = 0
Wynik: [1. 0. 0. 1.] Blad: 1.0
 
Dla eta = 0.01
Kombinacja: relu relu relu seed = 281
Wynik: [-0.00000000e+00  2.10942375e-15  1.22124533e-15  1.00000000e+00] Blad: 5.773159728050814e-15
Kombinacja: sigmoid relu relu seed = 242
Wynik: [-0.00000000e+00  8.76759554e-10  9.25838073e-10  9.99999998e-01] Blad: 3.4294886930297253e-09
Kombinacja: relu sigmoid relu seed = 4

Testy czasu

In [31]:
max_epochs = 10000
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 449 epoki = 2296
Kombinacja: relu relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu relu seed = 1 epoki = 10001
 
Dla eta = 0.01
Kombinacja: relu relu relu seed = 281 epoki = 582
Kombinacja: sigmoid relu relu seed = 242 epoki = 1585
Kombinacja: relu sigmoid relu seed = 449 epoki = 1662
Kombinacja: relu relu sigmoid seed = 191 epoki = 4407
Kombinacja: sigmoid sigmoid relu seed = 449 epoki = 8040
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.05
Kombinacja: relu sigmoid relu seed = 449 epoki = 330
Kombina

# Nauka OR

In [32]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

y = np.array([[0],[1],[1],[1]])

Testy dokladnosci

In [33]:
max_seed = 500
min_seed = 1
etas = np.array([0.001, 0.01, 0.05, 0.1, 0.5])
combinations = np.array([["relu", "relu", "relu"], ["relu", "relu", "sigmoid"], ["sigmoid", "sigmoid", "sigmoid"], ["sigmoid", "sigmoid", "relu"], 
                         ["relu", "sigmoid", "sigmoid"], ["relu", "sigmoid", "relu"],  ["sigmoid", "relu", "sigmoid"], ["sigmoid", "relu", "relu"]])
expected = np.array([[0.,1.,1.,1.]])
epochs = 5000
accuracy_tes(X, y, min_seed, max_seed, etas, combinations, expected, epochs) #  Czas - Okolo 15 min

Test dokladnosci 
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 418
Wynik: [0.0437007  0.97204968 0.96985588 1.0280084 ] Blad: 0.12980354455291665
Kombinacja: relu relu sigmoid seed = 79
Wynik: [0.5027705  0.99075809 0.99363459 0.99993964] Blad: 0.5184381861658528
Kombinacja: relu sigmoid relu seed = 449
Wynik: [0.38302054 0.83634434 0.87248285 0.99860307] Blad: 0.6755902777317042
Kombinacja: sigmoid relu relu seed = 132
Wynik: [0.47838557 0.83385314 0.77162587 1.00505553] Blad: 0.877962085971058
Kombinacja: sigmoid relu sigmoid seed = 493
Wynik: [0.99573923 0.99902527 0.99892726 0.99962226] Blad: 0.9981644332306013
Kombinacja: relu sigmoid sigmoid seed = 220
Wynik: [0.88138666 0.92821458 0.92302098 0.93209863] Blad: 1.0980524651875765
Kombinacja: sigmoid sigmoid sigmoid seed = 153
Wynik: [0.9320812  0.93736012 0.93619258 0.93949469] Blad: 1.1190338137815583
Kombinacja: sigmoid sigmoid relu seed = 132
Wynik: [0.71563082 0.76235007 0.76506163 0.79197228] Blad: 1.3962468504590713
 

Testy czasu

In [34]:
max_epochs = 10000
time_test(X, y, min_seed, max_seed, etas, combinations, expected, max_epochs)

Test czasu
 
Dla eta = 0.001
Kombinacja: relu relu relu seed = 418 epoki = 5472
Kombinacja: relu relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: relu sigmoid relu seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu relu seed = 1 epoki = 10001
 
Dla eta = 0.01
Kombinacja: relu relu relu seed = 147 epoki = 822
Kombinacja: relu sigmoid relu seed = 188 epoki = 2684
Kombinacja: relu relu sigmoid seed = 191 epoki = 3793
Kombinacja: sigmoid relu relu seed = 77 epoki = 4895
Kombinacja: sigmoid sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid sigmoid relu seed = 1 epoki = 10001
Kombinacja: relu sigmoid sigmoid seed = 1 epoki = 10001
Kombinacja: sigmoid relu sigmoid seed = 1 epoki = 10001
 
Dla eta = 0.05
Kombinacja: relu sigmoid relu seed = 188 epoki = 535
Kombinacj